In [1]:
#   @@@@@@@     @      @    @  @   #
#   @  @  @    @ @      @  @       #
#      @  @   @   @      @@    @   #
#      @     @@@@@@@    @  @   @   #
#      @    @       @  @    @  @   #

Привет, в этой практике мы с вами применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
<br>В занятии используется датасет собранный на основе данных <a href="https://www.kaggle.com/chicago/chicago-taxi-rides-2016">Chicago Taxi Rides 2016</a>
<br>Полная <a href="https://spark.apache.org/docs/latest/api/python/index.html">документация PySpark</a>.
<br>Схема данны:
<br>|-- taxi_id = идентификатор таксиста
<br>|-- trip_start_timestamp = время начала поездки
<br>|-- trip_end_timestamp = время окончания поездки
<br>|-- trip_seconds = время длительности поездки в секундах
<br>|-- trip_miles = мили проиденные во время поездки
<br>|-- fare = транспортные расходы
<br>|-- tips = назначенные чаевые
<br>|-- trip_total = общая стоимость поездки
<br>|-- payment_type = тип оплаты

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as spf

In [4]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [5]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")

In [6]:
spark

In [6]:
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=PySparkTasks>

Скачайте <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_data.parquet">taxi_data.parquet</a> и загрузите используя <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">SparkAPI</a>

In [8]:
df = spark.read.parquet('./taxi_data.parquet')

№1 Посчитайте количество загруженных строк.

In [15]:
df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
| 2540712|
+--------+



In [16]:
df.show(5)

+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare|tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|   5240| 2016-12-15 23:45:00|2016-12-16 00:00:00|         900|       2.5|10.75|2.45|      14.7| Credit Card|
|   1215| 2016-12-12 07:15:00|2016-12-12 07:15:00|         240|       0.4|  5.0| 3.0|       9.5| Credit Card|
|   3673| 2016-12-16 16:30:00|2016-12-16 17:00:00|        2400|      10.7| 31.0| 0.0|      31.0|        Cash|
|   5400| 2016-12-16 08:45:00|2016-12-16 09:00:00|         300|       0.0| 5.25| 2.0|      7.25| Credit Card|
|   1257| 2016-12-03 18:45:00|2016-12-03 18:45:00|         360|       0.3|  5.0| 0.0|       5.0|        Cash|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
only showi

Посмотрим схему данных:

In [17]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.corr">corr</a> & <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.cov">cov</a>

In [21]:
round(df.corr('trip_seconds', 'trip_miles'), 5)

0.4348

In [22]:
round(df.cov('trip_seconds', 'trip_miles'), 5)

2741.63542

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой. Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe">describe</a>

In [35]:
df.describe('trip_seconds', 'trip_miles').selectExpr('summary', 'round(trip_seconds, 1) as trip_seconds', 'round(trip_miles, 1) as trip_miles').show()

+-------+------------+----------+
|summary|trip_seconds|trip_miles|
+-------+------------+----------+
|  count|   2540178.0| 2540677.0|
|   mean|       801.0|       3.0|
| stddev|      1199.5|       5.3|
|    min|         0.0|       0.0|
|    max|     86399.0|     900.0|
+-------+------------+----------+



№4 Найдите самый НЕ популярный вид оплаты.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy">groupBy</a> <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [52]:
df.groupBy('payment_type').count().sort('count').show()

+------------+-------+
|payment_type|  count|
+------------+-------+
|    Way2ride|      3|
|       Pcard|    878|
|      Prcard|    968|
|     Dispute|   1842|
|     Unknown|   5180|
|   No Charge|  12843|
| Credit Card|1108843|
|        Cash|1410155|
+------------+-------+



№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [54]:
df.groupby('taxi_id').count().sort('count', ascending=False).show(3)

+-------+-----+
|taxi_id|count|
+-------+-----+
|    316| 2225|
|   6591| 2083|
|   5071| 2080|
+-------+-----+
only showing top 3 rows



№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.
<br> Подробней <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where">where</a>

In [57]:
df.where(df.payment_type == 'Cash').selectExpr('round(avg(trip_total), 5)').show()

+-------------------------+
|round(avg(trip_total), 5)|
+-------------------------+
|                 12.03526|
+-------------------------+



№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [58]:
df.where(df.trip_miles > 1000).selectExpr('count(distinct taxi_id)').show()

+-----------------------+
|count(DISTINCT taxi_id)|
+-----------------------+
|                      0|
+-----------------------+



№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [110]:
trip_seconds_max = df.agg({'trip_seconds': 'max'}).collect()[0][0]

In [111]:
df.where(f"trip_seconds = {trip_seconds_max}").selectExpr('trip_miles').show()

+----------+
|trip_miles|
+----------+
|       0.0|
|       0.0|
|       0.0|
+----------+



№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.
<br>Отсеките неизвестные машины (не определенный taxi_id).

In [113]:
df.dropna(subset='taxi_id').agg({''})

DataFrame[taxi_id: int, trip_start_timestamp: timestamp, trip_end_timestamp: timestamp, trip_seconds: int, trip_miles: double, fare: double, tips: double, trip_total: double, payment_type: string]

№10 Сколько поездок начиналось в самый загруженный час?
<br>Используйте функцию <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hour">hour</a>

In [19]:
from pyspark.sql.functions import hour

In [20]:
# Ваш код

№11 Сколько поездок началось во второй четверти дня?

In [21]:
# Ваш код

№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
<br> Ожидаемый формат дат YYYY-MM-DD
<br>Вам может понадобится конвертация типов <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast">cast</a>

In [ ]:
from pyspark.sql.types import DateType

In [ ]:
# Ваш код

№13 Сколько было заказов в дату с наибольшим спросом?

In [ ]:
# Ваш код

Подгрузите данные о марках машин из датасета <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_cars_data.parquet">taxi_cars_data.parquet</a>

In [ ]:
df_car = # Ваш код загрузки

In [ ]:
df_car.show()

№14 Какая марка машины самая распрастранненая среди таксистов?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split">split</a>

In [23]:
from pyspark.sql.functions import split

In [22]:
# Ваш код

№15 Сколько раз и какая модель машин чаще всего встречается в поездках?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">join</a>

In [ ]:
# Ваш код

Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [12]:
# Ваш код с coalesce(1)

Теперь загрузите данные из csv и проверьте типы методом printSchema().

In [13]:
# Ваш код с printSchema() для DataFrame из csv

Не забудьте посетить SparkUI и изучить историю ваших задач.

In [ ]:
spark